In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn
from torch import optim

In [3]:
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.flows.base import Flow
from nflows.distributions.dropout import StochasticDropout
from nflows.distributions.dropout import ProbabilityNet

In [4]:
# This works with any size x
def p(x, n_probs):
    sums = torch.sum(x, axis=1)
    probs = torch.cos(torch.ger(sums, torch.arange(1, n_probs+1, dtype=torch.float32)))**2
    norm = torch.sum(probs, axis=1)

    for i in range(n_probs):
        probs[:,i] /= norm
    
    return probs

In [5]:
def generate(n, drop_indices):
    n_probs = torch.max(drop_indices) + 1
    x = torch.rand(n, drop_indices.shape[0])
    probs = p(x, n_probs)

    # Pick a prob
    probs_cumsum = torch.cumsum(probs, axis=1)

    # Tensor with bools that are true when r passes the cumprob
    larger_than_cumprob = torch.rand(n,1) < probs_cumsum
    # Do the arange trick to find first nonzero
    # This is the HIGHEST LABEL FROM DROP_INDICES THAT IS KEPT
    selected_index = torch.argmax(larger_than_cumprob*torch.arange(n_probs, 0, -1), axis=1)

    '''
    print("The index of the selected probability")
    print("This is also the highest label in drop_indices that is kept")
    print(selected_index)
    ''' 
    
    # Find the index of the first true
    drop_mask = drop_indices > selected_index[:,None]
    x[drop_mask] = 0
    
    return x

In [6]:
'''
#Testing the probability net

drop_indices = torch.tensor([0,0,1,1,1,2,3,3,4])
n_dims = drop_indices.shape[0]
n_probs = torch.max(drop_indices).item() + 1
print(n_probs)

#prob_net = ProbabilityNet(n_dims, n_probs, 10*n_probs, 5)
prob_net = ProbabilityNet(n_dims, n_probs, 10*n_probs, 5)
optimizer = optim.Adam(prob_net.parameters())

iterations = 10000
batch_size = 128

for i in range(iterations):
    x = torch.rand(batch_size, drop_indices.shape[0])
    probs_real = p(x, n_probs)
    
    optimizer.zero_grad()
    loss = ((probs_real - prob_net(x))**2).mean()
    loss.backward()    
    optimizer.step()
    print(loss)
'''

'\n#Testing the probability net\n\ndrop_indices = torch.tensor([0,0,1,1,1,2,3,3,4])\nn_dims = drop_indices.shape[0]\nn_probs = torch.max(drop_indices).item() + 1\nprint(n_probs)\n\n#prob_net = ProbabilityNet(n_dims, n_probs, 10*n_probs, 5)\nprob_net = ProbabilityNet(n_dims, n_probs, 10*n_probs, 5)\noptimizer = optim.Adam(prob_net.parameters())\n\niterations = 10000\nbatch_size = 128\n\nfor i in range(iterations):\n    x = torch.rand(batch_size, drop_indices.shape[0])\n    probs_real = p(x, n_probs)\n    \n    optimizer.zero_grad()\n    loss = ((probs_real - prob_net(x))**2).mean()\n    loss.backward()    \n    optimizer.step()\n    print(loss)\n'

In [7]:
iterations = 10000
batch_size = 128

drop_indices = torch.tensor([0,0,1,1,1,2,3,3,4])
stoch_drop = StochasticDropout(drop_indices)
optimizer = optim.Adam(stoch_drop.parameters())

for i in range(iterations):
    x = generate(batch_size, drop_indices)
    loss = -stoch_drop.log_prob(x).mean()
    loss.backward()    
    optimizer.step()
    print(loss)

gBackward>)
tensor(1.6212, grad_fn=<NegBackward>)
tensor(1.6102, grad_fn=<NegBackward>)
tensor(1.6041, grad_fn=<NegBackward>)
tensor(1.6143, grad_fn=<NegBackward>)
tensor(1.6153, grad_fn=<NegBackward>)
tensor(1.5988, grad_fn=<NegBackward>)
tensor(1.6008, grad_fn=<NegBackward>)
tensor(1.6028, grad_fn=<NegBackward>)
tensor(1.5992, grad_fn=<NegBackward>)
tensor(1.5998, grad_fn=<NegBackward>)
tensor(1.6191, grad_fn=<NegBackward>)
tensor(1.6032, grad_fn=<NegBackward>)
tensor(1.6120, grad_fn=<NegBackward>)
tensor(1.6139, grad_fn=<NegBackward>)
tensor(1.6157, grad_fn=<NegBackward>)
tensor(1.6095, grad_fn=<NegBackward>)
tensor(1.6144, grad_fn=<NegBackward>)
tensor(1.6106, grad_fn=<NegBackward>)
tensor(1.5972, grad_fn=<NegBackward>)
tensor(1.6051, grad_fn=<NegBackward>)
tensor(1.6291, grad_fn=<NegBackward>)
tensor(1.6127, grad_fn=<NegBackward>)
tensor(1.6142, grad_fn=<NegBackward>)
tensor(1.6067, grad_fn=<NegBackward>)
tensor(1.6028, grad_fn=<NegBackward>)
tensor(1.6156, grad_fn=<NegBackward>)


KeyboardInterrupt: 

In [5]:
for i in range(iteration):
    print(i)

NameError: name 'iteration' is not defined

In [8]:
base_dist = BoxUniform(torch.zeros(4), torch.ones(4))
transforms = []
transforms.append(StochasticDropout(torch.tensor([1,2,3,4]), hidden_layers=5))
transform = CompositeTransform(transforms)
flow = Flow(transform, base_dist)
optimizer = optim.Adam(flow.parameters())

In [9]:
n_iterations = 10000
batch_size = 512

for iteration in range(n_iterations):
    x = generate(batch_size)

    optimizer.zero_grad()
    loss = -flow.log_prob(inputs=x).mean()
    loss.backward()
    optimizer.step()
    print(loss)

TypeError: generate() missing 1 required positional argument: 'drop_indices'

In [10]:
n_epochs = 5000
batch_size = 1
n_batches = m.ceil(x_data.shape[0]/batch_size)

for epoch in range(n_epochs):
    permutation = torch.randperm(x_data.shape[0])    

    # Loop over batches
    cum_loss = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, x_data.shape[0] )
        indices = permutation[batch_begin:batch_end]
        batch_x = x_data[indices]
        # Take a step
        optimizer.zero_grad()

        loss = -flow.log_prob(inputs=batch_x).mean()
        loss.backward()    
        optimizer.step()
    
        # Compute cumulative loss
        cum_loss = (cum_loss*batch + loss.item())/(batch+1)
        
    print("batch = ",batch+1, "/", n_batches, "loss = ", cum_loss)

NameError: name 'x_data' is not defined

In [11]:
with torch.no_grad():
    x_flow = flow.sample(10000)

ModuleAttributeError: 'StochasticDropout' object has no attribute 'inverse'

In [12]:
plt.hist(np.count_nonzero(x_data.numpy(), axis=1), np.linspace(-0.5,4.5,6), histtype='stepfilled', edgecolor="black", facecolor="lightgray", density=True)
plt.hist(np.count_nonzero(x_flow.numpy(), axis=1), np.linspace(-0.5,4.5,6), edgecolor="red", histtype="step", density=True)
plt.show()

NameError: name 'x_data' is not defined

In [13]:
x_random = torch.rand(10,4)

In [14]:
p(x_random)

TypeError: p() missing 1 required positional argument: 'n_probs'

RuntimeError: The size of tensor a (10) must match the size of tensor b (5) at non-singleton dimension 1

In [15]:
test = StochasticDropout(torch.tensor([1,2,3,4]), hidden_layers=2)

In [16]:
a = test.inverse(torch.rand(10,4))

ModuleAttributeError: 'StochasticDropout' object has no attribute 'inverse'